Enter the number of rows: 6
Enter the number of columns: 17


In [25]:
import heapq
import itertools

# Step 1: Create an array or grid of size defined by the user
n = 6 #int(input("Enter the number of rows: "))
m = 17 #int(input("Enter the number of columns: "))
grid = [[' ' for _ in range(m)] for _ in range(n)]

# Define obstacle locations
obstacles = [(2, 2), (5, 3), (4, 10), (0, 13), (2, 15)]

# Define robot initial positions
robot1_position = (0, 6)
robot2_position = (5, 9)

# Define pick-up cell locations for each robot
robot1_pick_up_cell = (5, 0)
robot2_pick_up_cell = (4, 6)

# Define delivery cell locations for each robot
robot1_delivery_cell = (0, 8)
robot2_delivery_cell = (5, 5)

# Define final destination locations for each robot
robot1_final_destination = (3, 12)
robot2_final_destination = (0, 10)

# Print the grid with obstacle and location markers
for i in range(n):
    for j in range(m):
        if (i, j) in obstacles:
            grid[i][j] = 'X'
        elif (i, j) == robot1_position:
            grid[i][j] = 'R1'
        elif (i, j) == robot2_position:
            grid[i][j] = 'R2'
        elif (i, j) == robot1_pick_up_cell:
            grid[i][j] = 'P1'
        elif (i, j) == robot2_pick_up_cell:
            grid[i][j] = 'P2'
        elif (i, j) == robot1_delivery_cell:
            grid[i][j] = 'D1'
        elif (i, j) == robot2_delivery_cell:
            grid[i][j] = 'D2'
        elif (i, j) == robot1_final_destination:
            grid[i][j] = 'F1'
        elif (i, j) == robot2_final_destination:
            grid[i][j] = 'F2'

# Print the grid
for row in grid:
    print(' '.join(row))


# Function to move the robot one cell at a time
def move_robot(start, end):
    path = []
    current_position = start

    while current_position != end:
        x_diff = end[0] - current_position[0]
        y_diff = end[1] - current_position[1]

        if x_diff > 0:
            next_position = (current_position[0] + 1, current_position[1])
        elif x_diff < 0:
            next_position = (current_position[0] - 1, current_position[1])
        elif y_diff > 0:
            next_position = (current_position[0], current_position[1] + 1)
        elif y_diff < 0:
            next_position = (current_position[0], current_position[1] - 1)

        if next_position not in obstacles:
            path.append(next_position)
            current_position = next_position
        else:
            break

    return path

# to find the shortest path
def astar(grid, start, end):
    def heuristic(node):
        return abs(node[0] - end[0]) + abs(node[1] - end[1])

    open_list = [(0, start)]
    came_from = {}
    g_score = {(i, j): float('inf') for i in range(n) for j in range(m)}
    g_score[start] = 0

    while open_list:
        current_g, current_cell = heapq.heappop(open_list)

        if current_cell == end:
            path = []
            while current_cell in came_from:
                path.append(current_cell)
                current_cell = came_from[current_cell]
            path.reverse()
            return path

        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            neighbor = (current_cell[0] + dx, current_cell[1] + dy)
            if (0 <= neighbor[0] < n and 0 <= neighbor[1] < m) and neighbor not in obstacles:
                tentative_g_score = g_score[current_cell] + 1
                if tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current_cell
                    g_score[neighbor] = tentative_g_score
                    f_score = tentative_g_score + heuristic(neighbor)
                    heapq.heappush(open_list, (f_score, neighbor))

    return None

# Define a class to represent agents
class Agent:
    def __init__(self, start, pick_up, delivery, destination):
        self.start = start
        self.pick_up = pick_up
        self.delivery = delivery
        self.destination = destination
        self.path = []

# Initialize agents
robot1_agent = Agent(robot1_position, robot1_pick_up_cell, robot1_delivery_cell, robot1_final_destination)
robot2_agent = Agent(robot2_position, robot2_pick_up_cell, robot2_delivery_cell, robot2_final_destination)

agents = [robot1_agent, robot2_agent]

# CBS algorithm
while True:
    for agent in agents:
        # Plan a path for each agent using A* pathfinding algorithm
        path_to_pick_up = astar(grid, agent.start, agent.pick_up)
        path_to_delivery = astar(grid, agent.pick_up, agent.delivery)
        path_to_destination = astar(grid, agent.delivery, agent.destination)

        agent.path = path_to_pick_up + path_to_delivery + path_to_destination

    conflict_resolution_needed = False

    for i, agent1 in enumerate(agents):
        for j, agent2 in enumerate(agents):
            if i != j and any(agent1.path[timestep] == agent2.path[timestep] for timestep in range(min(len(agent1.path), len(agent2.path)))):
                conflict_resolution_needed = True
                agent1.start = agent2.pick_up  # Simple conflict resolution, you can implement more sophisticated strategies

    if not conflict_resolution_needed:
        break

# Function to display the grid with paths
def display_grid_with_paths(grid, agents):
    for i, agent in enumerate(agents):
        marker = f'-R{i + 1}'
        for x, y in agent.path:
            grid[x][y] += marker

# Display the grid with paths
display_grid_with_paths(grid, agents)

# Print the final grid
print("Final Grid with Path")
for row in grid:
    print(' '.join(row))


            R1   D1   F2     X      
                                 
    X                         X  
                        F1        
            P2       X            
P1     X   D2       R2              
Final Grid with Path
 -R1-R1  -R1-R1  -R1-R1  -R1-R1  -R1-R1  -R1-R1-R2 R1-R1-R2  -R1-R2 D1-R1-R2  -R1-R2 F2-R1-R2  -R1  -R1 X      
 -R1-R1          -R2              -R1        
 -R1-R1   X      -R2              -R1     X  
 -R1-R1          -R2             F1-R1        
 -R1-R1          -R2-R2 P2-R2  -R2  -R2  -R2 X            
P1-R1     X   D2-R2       R2              


In [26]:
# Separate paths for Robot 1 and Robot 2
robot1_pickup_path = robot1_agent.path[:len(path_to_pick_up)]
robot1_delivery_path = robot1_agent.path[len(path_to_pick_up):len(path_to_pick_up) + len(path_to_delivery)]
robot1_destination_path = robot1_agent.path[len(path_to_pick_up) + len(path_to_delivery):]

robot2_pickup_path = robot2_agent.path[:len(path_to_pick_up)]
robot2_delivery_path = robot2_agent.path[len(path_to_pick_up):len(path_to_pick_up) + len(path_to_delivery)]
robot2_destination_path = robot2_agent.path[len(path_to_pick_up) + len(path_to_delivery):]

# Print out paths for Robot 1
print("Path for Robot 1:")
for position in robot1_pickup_path:
    print(position)


for position in robot1_delivery_path:
    print(position)

for position in robot1_destination_path:
    print(position)

# Print out paths for Robot 2
print("Path for Robot 2:")
for position in robot2_pickup_path:
    print(position)

for position in robot2_delivery_path:
    print(position)


for position in robot2_destination_path:
    print(position)


Path for Robot 1:
(0, 5)
(0, 4)
(0, 3)
(0, 2)
(0, 1)
(0, 0)
(1, 0)
(2, 0)
(3, 0)
(4, 0)
(5, 0)
(4, 0)
(3, 0)
(2, 0)
(1, 0)
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(1, 12)
(2, 12)
(3, 12)
Path for Robot 2:
(4, 9)
(4, 8)
(4, 7)
(4, 6)
(4, 5)
(5, 5)
(4, 5)
(3, 5)
(2, 5)
(1, 5)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
